In [1]:
import pandas as pd
import sys
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
df.shape

(7613, 5)

In [6]:
df['location'].value_counts()

USA                            104
New York                        71
United States                   50
London                          45
Canada                          29
                              ... 
Walthamstow, London              1
Finland                          1
Not where I want to be, yet      1
Hyderabad Telangana INDIA        1
?semekeepschanging@soyeh?        1
Name: location, Length: 3341, dtype: int64

In [7]:
df.drop('location', axis =1, inplace = True)


# almost 35% is null and the remaining 5k values are spead across 3k countries

# so, this feature seems to have very less ability to differentiate the target

In [8]:
df['text'].isnull().sum()

0

In [9]:
stopwords = set(stopwords.words("english"))

In [10]:
df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [11]:
corpus = []

### text feature

In [12]:
# word extraction

ps = PorterStemmer()

def word_extraction(df,col):

    for ix in range(0,df.shape[0]):
        s = ""
        for i in df[col][ix]:
            if i.isalpha() or i == ' ':
                s += i.lower()
        s = s.strip()
        df[col][ix] = s
    
# stop common words used

def stop_word(df,col):

    for ix in range(0,df.shape[0]):
        s = ""
        for i in word_tokenize(df[col][ix]):
            if i not in stopwords:
                s = s+" "+i
        s = s.strip()
        df[col][ix] = s
        corpus.append(s)

In [13]:
word_extraction(df,'text')
print(df['text'][0])
stop_word(df,'text')
print(df['text'][0])

C:\Users\ganeshramm\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


our deeds are the reason of this earthquake may allah forgive us all


C:\Users\ganeshramm\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


deeds reason earthquake may allah forgive us


In [14]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, min_df=5, max_df=5000)
X = cv.fit_transform(corpus).toarray()

In [15]:
y = df['target']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [17]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred, target_names = ['D','ND']))

              precision    recall  f1-score   support

           D       0.80      0.87      0.83       886
          ND       0.79      0.69      0.74       637

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [19]:
X.shape

(7613, 2500)

In [20]:
dff = pd.read_csv("test.csv")

In [21]:
corpus = []

In [22]:
word_extraction(dff,'text')
stop_word(dff,'text')

C:\Users\ganeshramm\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\ganeshramm\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
XX = cv.transform(corpus).toarray()

In [24]:
y_pred=spam_detect_model.predict(XX)

In [25]:
dic = {'id': dff['id'], 'target' : y_pred}

In [26]:
pd.DataFrame(dic).to_csv("result.csv")